In [11]:
!pip list

Package                                  Version
---------------------------------------- --------------
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.12
aiosignal                                1.3.2
aniso8601                                10.0.0
annotated-types                          0.7.0
anyio                                    4.7.0
argcomplete                              3.5.3
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asttokens                                3.0.0
async-lru                                2.0.4
attrs                                    24.2.0
autogen-agentchat                        0.4.5
autogen-core                             0.4.5
autogen-ext                              0.4.5
azure-ai-evaluation                      1.1.0
azure-ai-inference                       1.0.0b9
azure-ai-projects                        

In [2]:
# Import necessary libraries for creating and running AI agents with Semantic Kernel
# We're importing asyncio for asynchronous operations, dotenv for loading environment variables,
# OpenAI client for API access, and Semantic Kernel components for creating agents
import asyncio
import os
from dotenv import load_dotenv

from typing import Annotated
from openai import AsyncOpenAI


from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory


from semantic_kernel.agents.open_ai import OpenAIAssistantAgent
from semantic_kernel.contents import AuthorRole, ChatMessageContent
from semantic_kernel.functions import kernel_function


## Creating the Client and Kernel 

In this sample, we will use [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for access to the LLM. 

The `ai_model_id` is defined as `gpt-4o-mini`. Try changing the model to another model available on the GitHub Models marketplace to see the different results. 

For us to us the `Azure Inference SDK` that is used for the `base_url` for GitHub Models, we will use the `AsyncOpenAI` connector within Semantic Kernel. There are also other [available connectors](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) to use Semantic Kernel for other model providers.

We will also create a `Kernel`. A `kernel` is a collection of the services and plugins that will be used by your Agents. In this snipppet, we are creating the kernel and adding the `chat_completion_service` to it.  

In [3]:
# Set up OpenAI client with GitHub Models
# Load environment variables, create AsyncOpenAI client with GitHub token
# Configure Semantic Kernel with OpenAI chat completion service
load_dotenv()
client = AsyncOpenAI(
    api_key=os.getenv("GITHUB_TOKEN"), base_url="https://models.inference.ai.azure.com/")

kernel = Kernel()
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
    service_id="agent",
)
kernel.add_service(chat_completion_service)

## Creating the Agent 

Below we will are creating the Agent called `TravelAgent` and also creating a variable called `AGENT_INSTRUCTIONS`. We will later add this to our `system_message` that will give the agent instructions on the task, behavior and tone.

For this example, we are using very simple instructions. You can change these instructions to see how the agent responds differently. 

In [4]:
# Creating a travel agent with ChatCompletionAgent
# This agent will use the configured OpenAI service to generate responses
# about vacation planning based on the system instructions
AGENT_NAME = "TravelAgent"
AGENT_INSTRUCTIONS = "You are a travel agent that plans great vacations"
agent = ChatCompletionAgent(service_id="agent", kernel=kernel, name=AGENT_NAME)

## Running the Agents 

Now we can run the Agent by defining the `ChatHistory` and adding the `system_message` to it. We will use the `AGENT_INSTRUCTIONS` that we defined earlier. 

After these are defined, we create a `user_inputs` that will be what the user is sending to the agent. In this case, we have set this message to `Plan me a sunny vacation`. 

Feel free to change this message to see how the agent responds differently. 

In [5]:
# This cell runs the travel agent in an async function
# It creates a chat history, adds the system instructions,
# sends a user query about a Seoul vacation,
# and prints both user input and agent responses
async def main():
    # Define the chat history
    chat_history = ChatHistory()
    chat_history.add_system_message(AGENT_INSTRUCTIONS)

    user_inputs = [
        "Could you help me plan a sunny vacation in Seoul, South Korea?",
    ]
    for user_input in user_inputs:
        # Add the user input to the chat history
        chat_history.add_user_message(user_input)
        print(f"# User: '{user_input}'")
        # Invoke the agent to get a response
        try:
            async for content in agent.invoke(chat_history):
                # Add the response to the chat history
                chat_history.add_message(content)
                print(f"# Agent - {content.name or '*'}: '{content.content}'")
        except Exception as e:
            print(f"Error: {e}")

# For Jupyter notebooks, use this instead of asyncio.run():
await main()

# User: 'Could you help me plan a sunny vacation in Seoul, South Korea?'
# Agent - TravelAgent: 'Absolutely! Seoul, South Korea, offers a vibrant mix of traditional culture and modern excitement. While the weather can vary, late spring (April to June) and early fall (September to October) are usually the best times to visit for sunny weather. Here's a plan for a sunny vacation in Seoul:

### Duration: 5 Days

#### Day 1: Arrival and Relaxation
- **Morning:** Arrive in Seoul and check into a centrally-located hotel (consider areas like Myeongdong or Gangnam for shopping and dining).
- **Afternoon:** Take a leisurely stroll through **Namsan Park** and visit the iconic **N Seoul Tower** for panoramic views of the city.
- **Evening:** Explore **Myeongdong** for street food and shopping. Don’t miss the Tteokbokki (spicy rice cakes) and the various Korean snacks available.

#### Day 2: Historical Sightseeing
- **Morning:** Visit **Gyeongbokgung Palace**. Try to catch the changing of the guar